In [6]:
from csv import excel
from typing import Optional

from ccxt.base.types import Balances
from hyperliquid.ccxt.base.types import Order

dex = Dex('BTC', 'USDC')

In [8]:
print(dex.get_perp_available_balance())
print(dex.get_current_price())

1001.347451
111628.5


MODELS

In [3]:
from dataclasses import dataclass
from typing import Optional, List, Any

@dataclass
class Info:
    coin: str
    side: str
    limitPx: str
    sz: str
    oid: str
    timestamp: str
    triggerCondition: str
    isTrigger: bool
    triggerPx: str
    children: List[Any]
    isPositionTpsl: bool
    reduceOnly: bool
    orderType: str
    origSz: str
    tif: str
    cloid: Optional[str]

@dataclass
class Order:
    info: Info
    id: str
    clientOrderId: Optional[str]
    timestamp: int
    datetime: str
    lastTradeTimestamp: Optional[int]
    lastUpdateTimestamp: Optional[int]
    symbol: str
    type: str
    timeInForce: str
    postOnly: bool
    reduceOnly: bool
    side: str
    price: float
    triggerPrice: Optional[float]
    amount: float
    cost: float
    average: Optional[float]
    filled: float
    remaining: float
    status: str
    fee: Optional[Any]
    trades: List[Any]
    fees: List[Any]
    stopPrice: Optional[float]
    takeProfitPrice: Optional[float]
    stopLossPrice: Optional[float]



import dacite

def parse_order(api_data) -> Order:
    return dacite.from_dict(Order, api_data)


In [16]:
import ccxt
import dataclasses
from ccxt.base.types import Balances

def to_float(value) -> float:
    return float(value)

@dataclasses.dataclass
class AccountData:
    accountValue: float
    positionMargin: float

class Dex:
    buy = 'buy'
    sell = 'sell'

    def __init__(self, symbol: str, marginCoin: str):
        self.symbol = symbol
        self.marginCoin = marginCoin
        self.dex = ccxt.hyperliquid({
            'walletAddress': '0x765EaafC85566466EF63bc3D3e1f507526b6Cc82',
            "privateKey": "0x208d00493f51713bd0c42979e66180d38ff0e128198a07c3dbd5231a53f44791",
            'options': {'sandbox': True},
        })
        self.previous_orders = []

    def get_open_orders(self) -> List[Order]:
        open_orders = self.dex.fetch_open_orders()
        return [parse_order(order) for order in open_orders]

    def buy_at_market_price(self, amount: float, price: float) -> float:
        print(f"Buying at market price : {amount} at {price}")
        price = self.dex.create_order(self.get_symbol(), 'market', 'buy', amount, price)
        return to_float(price)

    def createLongLimit(self, qty, price) -> Order:
        print(f"Creating long limit order : {qty} at {price}")
        order = self.dex.create_order(self.get_symbol(), 'limit', 'buy', qty, price)
        return parsed_order(order)

    def createShortLimit(self, qty, price):
        print(f"Creating short limit order : {qty} at {price}")
        order = self.dex.create_order(self.get_symbol(), 'limit', 'sell', qty, price, params={'reduceOnly': True})
        return parsed_order(order)

    def get_perp_available_balance(self) -> float:
        amount = self.dex.fetch_balance()[self.marginCoin]['free']
        return to_float(amount)

    def get_perp_balance_infos(self) -> Balances:
        return self.dex.fetch_balance()

    def get_current_price(self) -> float:
        price = self.dex.fetch_ticker(self.get_symbol())['last']
        return to_float(price)

    def get_symbol(self) -> str:
        return self.symbol + '/' + self.marginCoin + ':' + self.marginCoin

    def get_account_data(self) -> AccountData :
        full_data = self.dex.fetch_balance()
        # nb: there is also a 'MarginSummary' key. Since we are in CrossMargin they contain the same data
        return AccountData(
            accountValue=float(full_data['info']['crossMarginSummary']['accountValue']),
            positionMargin=float(full_data['info']['crossMarginSummary']['totalMarginUsed']))

    def get_account_value(self) -> float:
        account_value = self.dex.fetch_balance()
        return to_float(account_value)

#
#       principe pour les positions
#
# - Etat initial :
#  - ouverture de 3 long sous le prix BTC actuel à des intervalles définis par le gap
#  - ouverture de 1 short au dessus du prix BTC actuel au même intervalle défini par le gap
#
# - quand un long est executé :
#   - ouverture d'une position long (acheteuse) en dessous (prix BTC actuel - gap)
#   - ouverture d'une position short (vendeuse) au dessus (prix BTC actuel + gap)
#
# - quand un short est executé :
#   - Idem


#   principe pour la quantité achetée sur un long :
#
# objectif : éloigner la liquidation
# - quantité adaptative
#   - PERP funds / 6 / prix BTC

from hyperliquid.ccxt.base.types import OrderSide
from typing import Optional

class Algo:

    GAPS = [500, 1000, 1500, 2000, 2500, 3000]
    currentGapIdx = 0
    maxLeverage = 40
    perpFundsPercentageForInitialLong = 10 # initial percentage to open Long position with PERP funds
    maxLongPositions = 3

    BUY: OrderSide = 'buy'
    SELL: OrderSide = 'sell'

    previous_orders = [Order]

    def getGap(self):
        return self.GAPS[self.currentGapIdx]

    def __init__(self, symbol: str, marginCoin: str):
        self.symbol = symbol
        self.marginCoin = marginCoin
        self.dex = Dex(symbol, marginCoin)

    def launch(self):
        total_amount = self.dex.get_perp_available_balance()
        current_price = self.dex.get_current_price()
        qty = total_amount / 6 / current_price

        print(f'Total amount: {total_amount}')
        print(f'Current price: {current_price}')
        print(f'Quantity: {qty}')

        # buy at market price
        self.dex.buy_at_market_price(amount=total_amount - 1, price=current_price)

        gap = self.getGap()

        # init short position
        short_price = current_price + gap
        order = self.dex.createShortLimit(qty, short_price)
        self.previous_orders.append(order)

        # init long positions
        for i in range(1, self.maxLongPositions):
            long_price = current_price - (i * gap)
            order = self.dex.createLongLimit(qty, long_price)
            self.previous_orders.append(order)

    def set_previous_orders(self):
        self.previous_orders = self.dex.get_open_orders()

    def check_orders(self):
        current_open_orders = self.dex.get_open_orders()
        previous_order_ids = [o.id for o in self.previous_orders]
        executed_orders = [order for order in current_open_orders if order.id not in previous_order_ids]

        ## DEMO
        if len(executed_orders) == 0:
            executed_orders = [current_open_orders[0]]
            executed_orders[0].price = 111000

        # TODO : check if many orders were executed. If true it could be more interesting to do something else
        if executed_orders:
            print(f"Executed orders: {executed_orders}")
            #asset_price = self.dex.get_current_price()
            asset_price = 110300
            position_qty = self.compute_position_qty(executed_orders[0], asset_price)

            if self.should_place_short(asset_price, current_open_orders):
                short_price = asset_price + self.getGap()
                print(f"Creating short limit order: {position_qty} at {short_price}")
                #self.dex.createShortLimit(position_qty, short_price)

            if self.should_place_long(asset_price, current_open_orders):
                long_price = asset_price - self.getGap()
                print(f"Creating long limit order: {position_qty} at {long_price}")
                #self.dex.createLongLimit(position_qty, long_price)



    def compute_position_qty(self, executed_order: dict, asset_price: float) -> float:
        print(executed_order)
        account_data = self.dex.get_account_data()
        return (account_data.accountValue - account_data.positionMargin) / 6 / asset_price

    def should_place_short(self, asset_price, orders: [Order]) -> bool:
        order = self.find_order_in_range(asset_price, orders, self.SELL, self.getGap())
        if order:
            print(f"Order already exists in range: {order.get('id') - order.get('price')}")
            return False
        return True

    def should_place_long(self, asset_price: float, orders: [Order]) -> bool:
        order = self.find_order_in_range(asset_price, orders, self.BUY, self.getGap())
        if order:
            print(f"Order already exists in range: {order.get('id') - order.get('price')}")
            return False
        return True

    def find_order_in_range(self, asset_price, orders: [Order], order_type: OrderSide, gap: float) -> Optional[any]:
        target_price = asset_price + gap
        price_range_start = target_price - (gap)
        price_range_end = target_price - (gap)

        order_in_range = list(filter(lambda o:
                                    order_type == o.side and
                                    price_range_start <= o.price <= price_range_end, orders))
        if len(order_in_range) > 0:
            return order_in_range[0]
        return None


algo = Algo('BTC', 'USDC')
algo.set_previous_orders()
algo.check_orders()

# Optionally, cancel all orders after execution
#self.reset_all_orders()

    #def reset_all_orders(self):
    #    open_orders = self.dex.dex.fetch_open_orders()
    #    for order in open_orders:
    #        self.dex.dex.cancel_order(order['id'])
    #    print("All orders cancelled.")

#algo = Algo('BTC', 'USDC')
#algo.launch()

Executed orders: [Order(info=Info(coin='BTC', side='A', limitPx='110000.0', sz='0.00612', oid='31003279435', timestamp='1748016113007', triggerCondition='N/A', isTrigger=False, triggerPx='0.0', children=[], isPositionTpsl=False, reduceOnly=True, orderType='Limit', origSz='0.01', tif='Gtc', cloid=None), id='31003279435', clientOrderId=None, timestamp=1748016113007, datetime='2025-05-23T16:01:53.007Z', lastTradeTimestamp=None, lastUpdateTimestamp=None, symbol='BTC/USDC:USDC', type='limit', timeInForce='GTC', postOnly=False, reduceOnly=True, side='sell', price=111000, triggerPrice=None, amount=0.01, cost=426.8, average=None, filled=0.00388, remaining=0.00612, status='open', fee=None, trades=[], fees=[], stopPrice=None, takeProfitPrice=None, stopLossPrice=None)]
Order(info=Info(coin='BTC', side='A', limitPx='110000.0', sz='0.00612', oid='31003279435', timestamp='1748016113007', triggerCondition='N/A', isTrigger=False, triggerPx='0.0', children=[], isPositionTpsl=False, reduceOnly=True, ord

In [7]:
algo = Algo('BTC', 'USDC')
algo.set_previous_orders()
algo.check_orders()

In [19]:
import requests

url = 'https://api.hyperliquid.xyz/info'
response = requests.post(url)
print(response)
#data = response.json()
#@print(data)


<Response [415]>


In [4]:
dex = Dex('BTC', 'USDC')
dex.get_perp_balance_infos()

{'info': {'marginSummary': {'accountValue': '846.372268',
   'totalNtlPos': '9397.69173',
   'totalRawUsd': '-8551.319462',
   'totalMarginUsed': '234.942293'},
  'crossMarginSummary': {'accountValue': '846.372268',
   'totalNtlPos': '9397.69173',
   'totalRawUsd': '-8551.319462',
   'totalMarginUsed': '234.942293'},
  'crossMaintenanceMarginUsed': '117.471146',
  'withdrawable': '0.0',
  'assetPositions': [{'type': 'oneWay',
    'position': {'coin': 'BTC',
     'szi': '0.08631',
     'leverage': {'type': 'cross', 'value': '40'},
     'entryPx': '109382.6',
     'positionValue': '9397.69173',
     'unrealizedPnl': '-43.126264',
     'returnOnEquity': '-0.1827225735',
     'liquidationPx': '100330.9467286745',
     'marginUsed': '234.942293',
     'maxLeverage': '40',
     'cumFunding': {'allTime': '120.155422',
      'sinceOpen': '120.155422',
      'sinceChange': '1.278508'}}}],
  'time': '1748001855650'},
 'USDC': {'total': 846.372268, 'used': 234.942293, 'free': 611.429975},
 'times

In [39]:
from enum import Enum

class InitialAlgoAction(Enum):
    LAUNCH = 'launch'
    RECOVER_PREVIOUS_STATE = 'recover_previous_state'

class AlgoPolling:
    def __init__(self, algo: Algo, initialAction: InitialAlgoAction, interval: int = 30):
        self.algo = algo
        if initialAction == InitialAlgoAction.LAUNCH:
            self.algo.launch()
        elif initialAction == InitialAlgoAction.RECOVER_PREVIOUS_STATE:
            self.algo.set_previous_orders()
        self.interval = interval

    def poll(self):
        import time
        while True:
            print("Checking orders...")
            self.algo.check_orders()
            time.sleep(self.interval)

In [40]:
algo = Algo('BTC', 'USDC')
AlgoPolling(algo, InitialAlgoAction.RECOVER_PREVIOUS_STATE, 30).poll()

Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Executed orders: []
Checking orders...
Execut

RequestTimeout: hyperliquid POST https://api.hyperliquid-testnet.xyz/info

UNIT TESTS

In [41]:
import pytest
from unittest.mock import Mock, patch
from datetime import datetime

@pytest.fixture
def mock_dex():
    mock = Mock()
    mock.get_perp_available_balance.return_value = 1000.0
    mock.get_current_price.return_value = 50000.0
    mock.dex.fetch_open_orders.return_value = [
        {
            'id': '1',
            'side': 'buy',
            'price': 49500.0,
            'amount': 0.1
        }
    ]
    return mock

@pytest.fixture
def algo_with_mock_dex(mock_dex):
    with patch('src.main.Dex') as mock_dex_class:
        mock_dex_class.return_value = mock_dex
        algo = Algo('BTC', 'USDC')
        return algo

def test_algo_polling_launch(algo_with_mock_dex):
    with patch('time.sleep') as mock_sleep:
        # Set up the mock to raise an exception after first iteration to stop the infinite loop
        mock_sleep.side_effect = [None, KeyboardInterrupt]

        polling = AlgoPolling(algo_with_mock_dex, InitialAlgoAction.LAUNCH)

        # Test the polling
        with pytest.raises(KeyboardInterrupt):
            polling.poll()

        # Verify initial launch was called
        algo_with_mock_dex.launch.assert_called_once()
        # Verify check_orders was called at least once
        assert algo_with_mock_dex.check_orders.call_count >= 1

def test_algo_polling_recover(algo_with_mock_dex):
    with patch('time.sleep') as mock_sleep:
        # Set up the mock to raise an exception after first iteration
        mock_sleep.side_effect = [None, KeyboardInterrupt]

        polling = AlgoPolling(algo_with_mock_dex, InitialAlgoAction.RECOVER_PREVIOUS_STATE)

        # Test the polling
        with pytest.raises(KeyboardInterrupt):
            polling.poll()

        # Verify set_previous_orders was called
        algo_with_mock_dex.set_previous_orders.assert_called_once()
        # Verify check_orders was called at least once
        assert algo_with_mock_dex.check_orders.call_count >= 1

TypeError: __main__.Order() got multiple values for keyword argument 'info'

In [59]:
dex = Dex('BTC', 'USDC')
data = dex.get_open_orders()
for order in data:
    print(order)

Order(info=Info(coin='BTC', side='A', limitPx='110000.0', sz='0.01', oid='31003279435', timestamp='1748016113007', triggerCondition='N/A', isTrigger=False, triggerPx='0.0', children=[], isPositionTpsl=False, reduceOnly=True, orderType='Limit', origSz='0.01', tif='Gtc', cloid=None), id='31003279435', clientOrderId=None, timestamp=1748016113007, datetime='2025-05-23T16:01:53.007Z', lastTradeTimestamp=None, lastUpdateTimestamp=None, symbol='BTC/USDC:USDC', type='limit', timeInForce='GTC', postOnly=False, reduceOnly=True, side='sell', price=110000.0, triggerPrice=None, amount=0.01, cost=0.0, average=None, filled=0.0, remaining=0.01, status='open', fee=None, trades=[], fees=[], stopPrice=None, takeProfitPrice=None, stopLossPrice=None)
Order(info=Info(coin='BTC', side='A', limitPx='111000.0', sz='0.01', oid='31003286796', timestamp='1748016121059', triggerCondition='N/A', isTrigger=False, triggerPx='0.0', children=[], isPositionTpsl=False, reduceOnly=True, orderType='Limit', origSz='0.01', t

In [54]:
for order in data:
    #print(order)
    parsed_order = parse_order(order)
    print(parsed_order)

Order(info=Info(coin='BTC', side='A', limitPx='110000.0', sz='0.01', oid='31003279435', timestamp='1748016113007', triggerCondition='N/A', isTrigger=False, triggerPx='0.0', children=[], isPositionTpsl=False, reduceOnly=True, orderType='Limit', origSz='0.01', tif='Gtc', cloid=None), id='31003279435', clientOrderId=None, timestamp=1748016113007, datetime='2025-05-23T16:01:53.007Z', lastTradeTimestamp=None, lastUpdateTimestamp=None, symbol='BTC/USDC:USDC', type='limit', timeInForce='GTC', postOnly=False, reduceOnly=True, side='sell', price=110000.0, triggerPrice=None, amount=0.01, cost=0.0, average=None, filled=0.0, remaining=0.01, status='open', fee=None, trades=[], fees=[], stopPrice=None, takeProfitPrice=None, stopLossPrice=None)
Order(info=Info(coin='BTC', side='A', limitPx='111000.0', sz='0.01', oid='31003286796', timestamp='1748016121059', triggerCondition='N/A', isTrigger=False, triggerPx='0.0', children=[], isPositionTpsl=False, reduceOnly=True, orderType='Limit', origSz='0.01', t